In [ ]:
from os.path import join
import json
from collections import OrderedDict

from time import time
import datetime
import random
import numpy as np
np.warnings.filterwarnings('ignore')
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pysal
import scipy as sc
import scipy.stats as sc_stats
from scipy.linalg import norm
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize

from scipy.stats import pearsonr, f_oneway, chi2_contingency, ks_2samp
from scipy.stats import entropy, normaltest, mode, kurtosis, skew, pearsonr, moment

In [ ]:
experiment_data_directory = '../experiment_data/'
input_file = join(experiment_data_directory, 'all_datasets.tsv')
output_file_name = join(experiment_data_directory, 'dataset_features.tsv')

In [ ]:
all_datasets = pd.read_table(input_file)

In [ ]:
distributions = [
    'norm',
    'lognorm',
    'expon',
    'powerlaw',
    'uniform',
    'chi2'
]

In [ ]:
def detect_outlier_one_sided_MAD(points, thresh=3.5):
    """
    From: https://stackoverflow.com/questions/22354094/pythonic-way-of-detecting-outliers-in-one-dimensional-observation-data
    
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
def madm(a):
    a = np.ma.array(a).compressed() # should be faster to not use masked arrays.
    med = np.median(a)
    return np.median(np.abs(a- med))

def get_shared_elements(v1, v2):
    return [e for e in v1 if v1 in v2]

def get_unique(li, preserve_order=False):
    if preserve_order:
        seen = set()
        seen_add = seen.add
        return [x for x in li if not (x in seen or seen_add(x))]
    else:
        return np.unique(li)

def get_list_uniqueness(l):
    if len(l):
        return len(np.unique(l)) / len(l)
    else:
        return None

def detect_unique_list(l, THRESHOLD=0.95):
    if len(l) and ((len(np.unique(l)) / len(l)) >= THRESHOLD):
        return True
    return False

def list_entropy(l):
    return entropy(pd.Series(l).value_counts() / len(l))

def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array = np.add(array, 0.0000001) # values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0]#number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient

def calculate_overlap(a_data, b_data):
    a_min, a_max = np.min(a_data), np.max(a_data)
    a_range = a_max - a_min
    b_min, b_max = np.min(b_data), np.max(b_data)
    b_range = b_max - b_min
    has_overlap = False
    overlap_percent = 0
    if (a_max >= b_min) and (b_min >= a_min):
        has_overlap = True
        overlap = (a_max - b_min)
    if (b_max >= a_min) and (a_min >= b_min):
        has_overlap = True
        overlap = (b_max - a_min)
    if has_overlap:
        overlap_percent = max(overlap / a_range, overlap / b_range)
    if ((b_max >= a_max) and (b_min <= a_min)) or ((a_max >= b_max) and (a_min <= b_min)):
        has_overlap = True
        overlap_percent = 1
    return has_overlap, overlap_percent

In [ ]:
_SQRT2 = np.sqrt(2)
def hellinger(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / _SQRT2

# Q Features

In [ ]:
distributions = [
    'norm',
    'lognorm',
    'expon',
    'powerlaw',
    'uniform',
    'chi2'
]

def extract_q_features(v):
    r = OrderedDict()
    sample_mean = np.mean(v)
    sample_median = np.median(v)
    sample_var = np.var(v)
    sample_min = np.min(v)
    sample_max = np.max(v)
    sample_std = np.std(v)
    q1, q25, q75, q99 = np.percentile(v, [0.01, 0.25, 0.75, 0.99])
    iqr = q75 - q25

    r['mean'] = sample_mean
    r['normalized_mean'] = sample_mean / sample_max
    r['median'] = sample_median
    r['normalized_median'] = sample_median / sample_max

    r['var'] = sample_var
    r['std'] = sample_std
    r['coeff_var'] = (sample_mean / sample_var) if sample_var else None
    r['min'] = sample_min
    r['max'] = sample_max
    r['range'] = r['max'] - r['min']
    r['normalized_range'] = (r['max'] - r['min']) / sample_mean if sample_mean else None

    r['entropy'] = None
    try:
        binned_values, bin_edges = get_binned_values(v)
        r['entropy'] = entropy(binned_values)
    except Exception as e: print(e, v)
    r['gini'] = gini(v)
    r['q25'] = q25
    r['q75'] = q75
    r['med_abs_dev'] = np.median(np.absolute(v - sample_median))
    r['avg_abs_dev'] = np.mean(np.absolute(v - sample_mean))
    r['quant_coeff_disp'] = (q75 - q25) / (q75 + q25)
    r['coeff_var'] = sample_var / sample_mean
    r['skewness'] = skew(v)
    r['kurtosis'] = kurtosis(v)
    r['moment_5'] = moment(v, moment=5)
    r['moment_6'] = moment(v, moment=6)
    r['moment_7'] = moment(v, moment=7)
    r['moment_8'] = moment(v, moment=8)
    r['moment_9'] = moment(v, moment=9)
    r['moment_10'] = moment(v, moment=10)

    # Outliers
    outliers_15iqr = np.logical_or(v < (q25 - 1.5*iqr), v > (q75 + 1.5*iqr))
    outliers_3iqr = np.logical_or(v < (q25 - 3*iqr), v > (q75 + 3*iqr))
    outliers_1_99 = np.logical_or(v < q1, v > q99)
    outliers_3std = np.logical_or(v < (sample_mean - 3*sample_std), v > (sample_mean + 3*sample_std))
    r['percent_outliers_15iqr'] = np.sum(outliers_15iqr) / len(v)
    r['percent_outliers_3iqr'] = np.sum(outliers_3iqr) / len(v)
    r['percent_outliers_1_99'] = np.sum(outliers_1_99) / len(v)
    r['percent_outliers_3std'] = np.sum(outliers_3std) / len(v)

    r['has_outliers_15iqr'] = np.any(outliers_15iqr)
    r['has_outliers_3iqr'] = np.any(outliers_3iqr)
    r['has_outliers_1_99'] = np.any(outliers_1_99)
    r['has_outliers_3std'] = np.any(outliers_3std)

    #####
    # Statistical Distribution
    #####

    # Normal
    r['normality_statistic'] = None
    r['normality_p'] = None
    r['is_normal_5'] = None
    r['is_normal_1'] = None
    if len(v) >= 8:
        normality_k2, normality_p = normaltest(v)
        r['normality_statistic'] = normality_k2
        r['normality_p'] = normality_p
        r['is_normal_5'] = (normality_p < 0.05)
        r['is_normal_1'] = (normality_p < 0.01)

    # v = random.sample(list(v), min(len(v), num_samples))
    for dist_name in distributions:
        dist_function = getattr(sc_stats, dist_name)
        statistic, p_value = sc_stats.kstest(v, dist_name, args=dist_function.fit(v))
        r['{}_ks_statistic'.format(dist_name)] = statistic
        r['{}_ks_p'.format(dist_name)] = p_value
   
    
    sorted_v = np.sort(v)
    sequence_incremental_division = np.divide(sorted_v[:-1], sorted_v[1:])
    sequence_incremental_subtraction = np.diff(sorted_v)
    
    r['is_monotonic'] = np.all(sequence_incremental_subtraction <= 0) or np.all(sequence_incremental_subtraction >= 0)
    r['sortedness'] = np.absolute(pearsonr(v, sorted_v)[0])  # or use inversions
    r['is_sorted'] = np.array_equal(sorted_v, v)  # np.allclose(v, sorted_v)  # np.array_equal(sorted_v, v)

    r['lin_space_sequence_coeff'] = np.std(sequence_incremental_subtraction) / np.mean(sequence_incremental_subtraction)
    r['log_space_sequence_coeff'] = np.std(sequence_incremental_division) / np.mean(sequence_incremental_division)
    r['is_lin_space'] = r['lin_space_sequence_coeff'] <= 0.001
    r['is_log_space'] = r['log_space_sequence_coeff'] <= 0.001

    unique_elements = get_unique(v)
    r['num_unique_elements'] = unique_elements.size
    r['unique_percent'] = (r['num_unique_elements'] / len(v))
    r['is_unique'] = (r['num_unique_elements'] == len(v))

    return r

In [ ]:
def extract_c_features(v):
    r = OrderedDict()
    unique_elements = get_unique(v)
    r['num_unique_elements'] = unique_elements.size
    r['unique_percent'] = (r['num_unique_elements'] / len(v))
    r['is_unique'] = (r['num_unique_elements'] == len(v))
    
    r['entropy'] = list_entropy(v)

    value_counts = pd.Series(v).value_counts()
    r['percentage_of_mode'] = (value_counts.max() / len(v))
    r['same_num_per_group'] = (len(set(value_counts)) == len(value_counts))
    r['min_per_group'] = np.min(value_counts)
    r['max_per_group'] = np.max(value_counts)
    r['mean_per_group'] = np.mean(value_counts)
    r['std_per_group'] = np.std(value_counts)
    
    sorted_v = np.sort(v)
    r['is_sorted'] = np.array_equal(sorted_v, v)
    return r
    
def extract_qq_features(q1_values, q2_values):
    r = OrderedDict()
    df = pd.DataFrame({ 'W': q1_values, 'Z': q2_values })
    
    correlation_value, correlation_p = pearsonr(q1_values, q2_values)
    ks_statistic, ks_p = ks_2samp(q1_values, q2_values)
    has_overlap, overlap_percent = calculate_overlap(q1_values, q2_values)
        
    r['correlation_value'] = correlation_value
    r['correlation_abs_value'] = np.abs(correlation_value)
    r['correlation_p'] = correlation_p
    r['correlation_significant_005'] = (correlation_p < 0.05)

    r['ks_statistic'] = ks_statistic
    r['ks_p'] = ks_p
    r['ks_significant_005'] = (ks_p < 0.05)

    r['has_overlap'] = has_overlap
    r['overlap_percent'] = overlap_percent
    
    q_points = np.array(list(zip(q1_values, q2_values)))
    outliers = detect_outlier_one_sided_MAD(q_points)
    num_outliers = len([ o for o in outliers if o ])
    percent_outliers = num_outliers / df.shape[0]
    r['has_outliers_one-sided-MAD'] = any(outliers)
    r['num_outliers_one-sided-MAD'] = num_outliers
    r['percent_outliers_one-sided-MAD'] = percent_outliers
    
    r['entropy'] = None
    r['morans_i_value'] = None
    r['morans_i_p'] = None
    try:
        _, W_bin_edges = get_binned_values(df['W'])
        _, Z_bin_edges = get_binned_values(df['Z'])
        count_by_bins = df.groupby([pd.cut(df['W'], W_bin_edges, include_lowest=True), pd.cut(df['Z'], Z_bin_edges, include_lowest=True)]).count()['W'].fillna(0)
        count_by_bins_array = count_by_bins.values
        r['entropy'] = entropy(count_by_bins_array)

        count_by_bins_matrix = count_by_bins.unstack().as_matrix()
        w = pysal.lat2W(count_by_bins_matrix.shape[0], count_by_bins_matrix.shape[1])
        mi = pysal.Moran(count_by_bins_matrix, w)
        r['morans_i_value'] = mi.I
        r['morans_i_p'] = mi.p_norm
    except Exception as e: 
        print('QQ entropy and moran:', e, v)

    return r

def extract_cq_features(c_values, q_values):
    r = OrderedDict()
    df = pd.DataFrame({ 'C': c_values, 'Q': q_values })
    
    unique_c_field_values = get_unique(c_values)
    group_values = [ df[df['C'] == v]['Q'] for v in unique_c_field_values ]
    anova_result = f_oneway(*group_values)  

    r['one_way_anova_statistic'] = anova_result.statistic
    r['one_way_anova_p'] = anova_result.pvalue
    r['one_way_anova_significant_005'] = (anova_result.pvalue < 0.05)

    r['entropy'] = None
    r['morans_i_value'] = None
    r['morans_i_p'] = None
    try:
        _, bin_edges = get_binned_values(q_values, normalize=False)
        count_by_bins = df.groupby(['C', pd.cut(q_values, bin_edges, include_lowest=True)]).count()['Q'].fillna(0)
        count_by_bins_array = count_by_bins.values   
        r['entropy'] = entropy(count_by_bins_array)

        count_by_bins_matrix = count_by_bins.unstack().as_matrix()
        w = pysal.lat2W(count_by_bins_matrix.shape[0], count_by_bins_matrix.shape[1])
        mi = pysal.Moran(count_by_bins_matrix, w)
        r['morans_i_value'] = mi.I
        r['morans_i_p'] = mi.p_norm
    except Exception as e:
        print('CQ entropy and moran:', e, v)
    return r

# Global clusteredness
# 1) Find distances from mean point
# 2) Remove mean from distances
# 3) Divide demeaned distances by standard deviation
def get_global_clusteredness(v1, v2):
    combined_v = np.array(list(zip(v1, v2)))
    mean_point = np.mean(combined_v, axis=0)
    distances = [ np.linalg.norm([e, mean_point]) for e in combined_v ]
    z_score_distances = np.abs((distances - np.mean(distances)) / np.std(distances))

    z_score_distances = [ x for x in z_score_distances if not pd.isnull(x) ] 
    clusteredness = np.sum(z_score_distances)
    return clusteredness / v1.size

def normalize(v):
    return (v - np.min(v)) / (np.max(v) - np.min(v))

def get_binned_values(v, bins=20, normalize=True):
    hist_values, bin_edges = np.histogram(v, bins=bins, density=False)
    if normalize:
        hist_values = hist_values / hist_values.sum()
    return hist_values, bin_edges

def extract_cqq_features(c_values, w_values, z_values):
    r = OrderedDict()
    
    df = pd.DataFrame({
        'C': c_values,
        'W': w_values,
        'Z': z_values
    })
    
    df.dropna(how='any', inplace=True)
    
    normalized_w_values = normalize(w_values)
    normalized_z_values = normalize(z_values)
    df['W'] = normalized_w_values
    df['Z'] = normalized_z_values
    
    group_means = []
    clusteredness_by_group = []
    hellingers_distances = []
    for name, group in df.groupby('C'):
        means = np.mean(group)
        w_values, z_values = group['W'], group['Z']
        
        # Probability distributions
        try:
            w_normalized_values, _= get_binned_values(w_values)
            z_normalized_values, _ = get_binned_values(z_values)

            hellinger_d = hellinger(w_normalized_values, z_normalized_values)
            hellingers_distances.append(hellinger_d)

            clusteredness_by_group.append(get_global_clusteredness(w_values, z_values))
            group_means.append([np.mean(w_values), np.mean(z_values)])
        except Exception as e:
            print('Error calculating hellingers', e)
            continue
    
    group_means_df = pd.DataFrame(group_means, columns=['W', 'Z'])

    global_clusteredness = 1 - get_global_clusteredness(normalized_w_values, normalized_z_values)
    group_mean_clusteredness = 1 - get_global_clusteredness(group_means_df['W'], group_means_df['Z'])
    mean_of_group_clusteredness = 1 - np.mean(clusteredness_by_group)
    
#     sns.scatterplot(x='W', y='Z', hue='C', data=df)
#     plt.show()
#     print('Global', global_clusteredness)
#     print('Group mean', group_mean_clusteredness)
#     print('Mean of group', mean_of_group_clusteredness)

    r['global_clusteredness'] = global_clusteredness
    r['group_mean_clusteredness'] = group_mean_clusteredness
    r['mean_of_group_clusteredness'] = mean_of_group_clusteredness
    
    
    r['entropy'] = None
    try:
        _, W_bin_edges = get_binned_values(df['W'])
        _, Z_bin_edges = get_binned_values(df['Z'])
        count_by_bins = df.groupby([pd.cut(df['W'], W_bin_edges, include_lowest=True), pd.cut(df['Z'], Z_bin_edges, include_lowest=True)]).count()['W'].fillna(0)
        count_by_bins_array = count_by_bins.values
        r['entropy'] = entropy(count_by_bins_array)
    except Exception as e: 
        print('CQQ entropy and moran:', e, v)

    r['mean_hellingers'] = np.mean(hellingers_distances)
    r['min_hellingers'] = np.min(hellingers_distances)
    r['max_hellingers'] = np.max(hellingers_distances)
    r['std_hellingers'] = np.std(hellingers_distances)
    return r

In [ ]:
write_header = True
num_datasets = all_datasets.shape[0]

batch_size = 10
batch_features = []

start_time = time()
for i, row in all_datasets.iterrows():
    i = i + 1
    features = OrderedDict()
    corpus = row['corpus']
    dataset_id_with_combination_number = row['dataset_id']
    dataset_id = dataset_id_with_combination_number.rsplit('__')[0]
    df = pd.read_json(row['data'])
    
    C = np.ma.array(df['C']).compressed()
    W = np.ma.array(df['W']).compressed()
    Z = np.ma.array(df['Z']).compressed()
    
    features['corpus'] = corpus
    features['dataset_id'] = dataset_id
    features['dataset_id_with_combination_number'] = dataset_id_with_combination_number
    
    features['n_rows'] = df.shape[0]
    
    try:
        r = extract_c_features(C)
        for k, v in r.items(): features['c_{}'.format(k)] = v

        r = extract_q_features(W)
        for k, v in r.items(): features['q1_{}'.format(k)] = v

        r = extract_q_features(Z)
        for k, v in r.items(): features['q2_{}'.format(k)] = v

        r = extract_cq_features(C, W)
        for k, v in r.items(): features['cq1_{}'.format(k)] = v

        r = extract_cq_features(C, Z)
        for k, v in r.items(): features['cq2_{}'.format(k)] = v

        r = extract_qq_features(W, Z)
        for k, v in r.items(): features['qq_{}'.format(k)] = v

        r = extract_cqq_features(C, W, Z)
        for k, v in r.items(): features['cqq_{}'.format(k)] = v
            
        batch_features.append(features)
    except Exception as e:
        print('Uncaught high-level error:', e)
        continue
    
    if i % batch_size == 0:
        print('({}/{})'.format(i, num_datasets))
        elapsed_time = time() - start_time
        time_per_dataset = elapsed_time / i
        time_to_finish = (num_datasets * time_per_dataset) - elapsed_time
        print('Time to completion: {}'.format(datetime.timedelta(seconds=time_to_finish)))
        
        batch_features_df = pd.DataFrame(batch_features)
        batch_features_df.to_csv(output_file_name, sep='\t', mode='a', index=False, header=write_header)
        print(batch_features_df)
        batch_features = []
        write_header=False

In [ ]:
pd.DataFrame(all_features)

In [ ]:
df = pd.DataFrame({
    'W': np.random.normal(size=1000),
    'Z': np.random.normal(size=1000)
})

sns.scatterplot(x='W', y='Z', data=df)

_, W_bin_edges = get_binned_values(df['W'])
_, Z_bin_edges = get_binned_values(df['Z'])
grouped_by_bins = df.groupby([pd.cut(df['W'], W_bin_edges, include_lowest=True), pd.cut(df['Z'], Z_bin_edges, include_lowest=True)])
count_by_bins_matrix = grouped_by_bins.count()['W'].fillna(0).unstack().as_matrix()
w = pysal.lat2W(count_by_bins_matrix.shape[0], count_by_bins_matrix.shape[1])
mi = pysal.Moran(count_by_bins_matrix, w)
print(mi.I, mi.EI, mi.p_norm)

In [ ]:
df = pd.DataFrame({
    'W': np.random.uniform(size=1000),
    'Z': np.random.uniform(size=1000)
})

sns.scatterplot(x='W', y='Z', data=df)

_, W_bin_edges = get_binned_values(df['W'])
_, Z_bin_edges = get_binned_values(df['Z'])
grouped_by_bins = df.groupby([pd.cut(df['W'], W_bin_edges, include_lowest=True), pd.cut(df['Z'], Z_bin_edges, include_lowest=True)])
count_by_bins_matrix = grouped_by_bins.count()['W'].fillna(0).unstack().as_matrix()
w = pysal.lat2W(count_by_bins_matrix.shape[0], count_by_bins_matrix.shape[1])
mi = pysal.Moran(count_by_bins_matrix, w)
print(mi.I, mi.EI, mi.p_norm)

In [ ]:
df = pd.DataFrame({
    'C': [ 'a', 'b', 'b', 'b', 'b', 'c'],
    'Q': [ 1, 2, 2, 2, 2, 3]
})

q1_values = df['Q']
_, bin_edges = get_binned_values(q1_values, normalize=False)
count_by_bins = df.groupby(['C', pd.cut(q1_values, bin_edges, include_lowest=True)]).count()['Q'].fillna(0)
count_by_bins_array = count_by_bins.values
count_by_bins_matrix = count_by_bins.unstack().as_matrix()
print(count_by_bins.values)